In [ ]:
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np

In [ ]:
pd.set_option("max_colwidth", 100)
pd.set_option("display.max_rows",10)
pd.set_option("display.max_columns",5)
DS= pd.read_pickle('balanced_ai2_arc_challenge.pkl');
answer_data = DS
answer_data

In [ ]:
!pip install --upgrade openai

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key='your api key',
)

Choose the desired model: so only one of the next two cells shold run

In [ ]:
def get_completion(messages, model="gpt-4o"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:
def get_completion(messages, model="gpt-3.5-turbo-0125"):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

In [ ]:


import time

def connect_with_api(messages):
    max_retries = 3
    retries = 0

    while retries < max_retries:
        try:
            response = get_completion(messages)
            return response
        except Exception as e:
            print(f"An error occurred: {e}")
            retries += 1
            print(f"Retrying... (Attempt {retries} of {max_retries})")
            time.sleep(1)  # Wait for 1 second before retrying

    print("Failed to connect with the API after multiple attempts.")

In [ ]:
test_samples_per_answer = 199
train_samples_per_answer = 1
test_df = pd.DataFrame()
train_df = pd.DataFrame()
for answer in answer_data['answer'].unique():
    answer_df = answer_data[answer_data['answer'] == answer]
    test_samples = answer_df.sample(test_samples_per_answer, random_state=1)
    train_samples = answer_df.drop(test_samples.index).sample(train_samples_per_answer, random_state=1)

    test_df = pd.concat([test_df, test_samples])
    train_df = pd.concat([train_df, train_samples])

# Reset the index of the resulting DataFrames
test_df = test_df.reset_index(drop=True)
train_df = train_df.reset_index(drop=True)

In [ ]:
def clean_response(response):
    response = response.strip()
    if 'A' in response:
        return 'A'
    elif 'B' in response:
        return 'B'
    elif 'C' in response:
        return 'C'
    elif 'D' in response:
        return 'D'
    else:
        return None

In [ ]:

# Experiment configurations
def run_experiment(prompt_design, test_df, train_df, experiment_name):
    test_results = []
    for index, row in test_df.iterrows():
        if prompt_design == "zeroU":
            messages = [{"role": "user", "content": f"""Determine the answer of the example based on the provided question:
             For the question provided, classify its answer as a single Capital letter (without other marks or words like 'answer:'), either "A", "B", "C", or "D".
```{row['question']}```""" }]
        elif prompt_design == "zeroSU":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
             For the question provided, classify its answer as a single Capital letter (without other marks or words like 'answer:'), either "A", "B", "C", or "D"."""},
                        {"role": "user", "content": f"""```{row['question']}```"""}]
        elif prompt_design == "fewU":
            messages = [{"role": "user", "content": f"""Determine the answer of the example based on the provided question:
             For the question provided, classify its answer as a single Capital letter (without other marks or words like 'answer:'),either "A", "B", "C", or "D".
Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[1]['question']}``` - {train_df.iloc[1]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
...
```{row['question']}```""" }]
        elif prompt_design == "fewSU":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
             For the question provided, classify its answer as a single Capital letter (without other marks or words like 'answer:'), either "A", "B", "C", or "D"."""},
                        {"role": "user", "content": f"""Examples:
```{train_df.iloc[0]['question']}``` - {train_df.iloc[0]['answer']}
```{train_df.iloc[1]['question']}``` - {train_df.iloc[1]['answer']}
```{train_df.iloc[2]['question']}``` - {train_df.iloc[2]['answer']}
...
```{row['question']}```"""}]
        elif prompt_design == "fewSUA":
            messages = [{"role": "system", "content": f"""Determine the answer of the example based on the provided question.
            For the question provided, classify its answer as a single Capital letter (without other marks or words like 'answer:'), either either "A", "B", "C", or "D"."""},
                        {"role": "user", "content": train_df.iloc[0]['question']}, {"role": "assistant", "content": train_df.iloc[0]['answer']},
                        {"role": "user", "content": train_df.iloc[1]['question']}, {"role": "assistant", "content": train_df.iloc[1]['answer']},
                        {"role": "user", "content": train_df.iloc[2]['question']}, {"role": "assistant", "content": train_df.iloc[2]['answer']},
                        {"role": "user", "content": f"""```{row['question']}```"""}]

        prediction = connect_with_api(messages)
        print(index, ' ', prediction)

        test_results.append({'Index': index, 'question': row['question'], 'Predicted_answer': prediction, 'Actual_answer': row['answer']})

    results_df = pd.DataFrame(test_results)
    # Calculate structural accuracy before cleaning
    valid_answer = ['A', 'B', 'C', 'D']
    results_df['Is_Structurally_Valid'] = results_df['Predicted_answer'].apply(lambda x: x.strip().upper() in valid_answer)
    structural_accuracy = results_df['Is_Structurally_Valid'].mean()

    # Calculate F1 Score
    results_df['Predicted_answer'] = results_df['Predicted_answer'].apply(clean_response)
    results_df = results_df.dropna(subset=['Predicted_answer', 'Actual_answer'])
    f1 = f1_score(results_df['Actual_answer'], results_df['Predicted_answer'], average='micro')

    print(f"Experiment: {experiment_name}")
    print(f"The F1 Score for the predicted answer is: {f1}")
    print(f"The Structural Accuracy is: {structural_accuracy}")

    results_df.to_csv(f"/content/drive/MyDrive/Colab Notebooks/In_context_learning/{experiment_name}_results.csv", index=False)
    print(f"Results for {experiment_name} saved to {experiment_name}_results.csv")

In [ ]:
# Running the experiments
run_experiment("zeroU", test_df, train_df, "Zero-shot User Prompt")


In [ ]:
run_experiment("zeroSU", test_df, train_df, "Zero-shot System and User Prompt")


In [ ]:
run_experiment("fewU", test_df, train_df, "Few-shot User Prompt")


In [ ]:
run_experiment("fewSU", test_df, train_df, "Few-shot System and User Prompt")


In [ ]:
run_experiment("fewSUA", test_df, train_df, "Few-shot System, User, and Assistant Prompt")